# Tool to compare two mapper graphs

By selecting nodes on the graph on the left, the noded on the graph on the right are colored by the percentage of their covered points that in the selected nodes.

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import pickle

In [ ]:
from bokeh.io import show, save
from bokeh.plotting import figure

from bokeh.layouts import layout, column, row, grid
from bokeh.models import (BoxZoomTool, Circle, HoverTool,
                          MultiLine, Plot, Range1d, ResetTool,
                          BoxSelectTool,
                          ColumnDataSource, LabelSet,
                          TapTool, WheelZoomTool, PanTool,
                          ColorBar, LinearColorMapper, BasicTicker,
                          Button, TextInput,
                          CustomJS, MultiChoice,
                          SaveTool)

from bokeh.events import Tap, SelectionGeometry

from bokeh.plotting import from_networkx

from matplotlib import cm
from matplotlib.colors import to_hex

## Read the graphs

Each graph must be rapresented by an adjecency list (space separated)  
We assume nodes are numbered from 1 to N  
  
The list of points covereb by each node is a file with N lines, each line contains the points id (space separated) 

In [ ]:
# define the color palette
my_red_palette = cm.get_cmap(name='Reds')

# create and array with 101 colors
color_list = [to_hex(my_red_palette(i/100)) for i in range(101)]  

In [ ]:
# Prepare Data

TITLE_1 = 'Alexander up to 15 crossings. Select ball(s) to identify'
TITLE_2 = 'Jones up to 15 crossings. Click COLOR to see the selected balls on this graph'

###########
# GRAPH 1 #
###########

with open('../EqBM/pkl/alexander_bm.pkl', 'rb') as f:
    G1 = pickle.load(f)

###########
# GRAPH 2 #
###########
                          
with open('../EqBM/pkl/jones_bm.pkl', 'rb') as f:
    G2 = pickle.load(f)

for node in G2.nodes:
    G2.nodes[node]['coverage'] = 0

In [ ]:
G1.nodes[0].keys()

In [ ]:
G2.nodes[0].keys()

## UI

In [ ]:

##########
# PLOT 1 #
##########

plot1 = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1),
            sizing_mode="stretch_both",
            title=TITLE_1)

node_hover_tool = HoverTool(tooltips=[("index", "@index"), ("size", "@size")])
plot1.add_tools(PanTool(), node_hover_tool, BoxZoomTool(), WheelZoomTool(),
                ResetTool(), TapTool(), BoxSelectTool(), SaveTool())

graph_renderer_1 = from_networkx(G1, nx.spring_layout,
                                 seed=66, scale=1, center=(0, 0),
                                 k= 10/np.sqrt(len(G1.nodes)),
                                 iterations=2000)


## labels
# get the coordinates of each node
x_1, y_1 = zip(*graph_renderer_1.layout_provider.graph_layout.values())

# create a dictionary with each node position and the label
source_1 = ColumnDataSource({'x': x_1, 'y': y_1,
                             'node_id': [node for node in G1.nodes]})
labels_1 = LabelSet(x='x', y='y', text='node_id', source=source_1,
                    text_color='black', text_alpha=1, visible=False)

# nodes
graph_renderer_1.node_renderer.glyph = Circle(size='size rescaled',
                                            fill_color='color',
                                            fill_alpha=0.8)

# edges
graph_renderer_1.edge_renderer.glyph = MultiLine(line_color='black',
                                               line_alpha=0.8, line_width=1)

plot1.renderers.append(graph_renderer_1)
plot1.renderers.append(labels_1)

##########
# PLOT 2 #
##########

plot2 = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1),
            sizing_mode="stretch_both",
            title=TITLE_2)

node_hover_tool = HoverTool(tooltips=[("index", "@index"), ("size", "@size"),
                                      ("coverage", "@{coverage}{%0f}")])
plot2.add_tools(PanTool(), node_hover_tool, BoxZoomTool(), WheelZoomTool(),
                ResetTool(), SaveTool())

graph_renderer_2 = from_networkx(G2, nx.spring_layout,
                                 seed=42, scale=1, center=(0, 0),
                                 k= 10/np.sqrt(len(G2.nodes)),
                                 iterations=2000)

## labels
# get the coordinates of each node
x_2, y_2 = zip(*graph_renderer_2.layout_provider.graph_layout.values())

# create a dictionary with each node position and the label
source_2 = ColumnDataSource({'x': x_2, 'y': y_2,
                           'node_id': [node for node in G2.nodes]})
labels_2 = LabelSet(x='x', y='y', text='node_id', source=source_2,
                    text_color='black', text_alpha=1, visible=False)

# nodes
graph_renderer_2.node_renderer.glyph = Circle(size='size rescaled',
                                              fill_color='color',
                                              fill_alpha=0.8)

# edges
graph_renderer_2.edge_renderer.glyph = MultiLine(line_color='black',
                                                 line_alpha=0.8, line_width=1)

# color bar legend
color_mapper_2 = LinearColorMapper(palette=[to_hex(my_red_palette(color_id)) 
                                            for color_id in np.linspace(0, 1, 101)], 
                                   low=1, high=100)
color_bar_2 = ColorBar(color_mapper=color_mapper_2, ticker=BasicTicker(),
                       label_standoff=12, border_line_color=None, location=(0,0),
                       title='Percentage')

plot2.add_layout(color_bar_2, 'right')

plot2.renderers.append(graph_renderer_2)
plot2.renderers.append(labels_2)

################
# color button #
################

color_button = Button(label='COLOR',
                height_policy='fit',
                button_type="success")


################
# labels button #
################

labels_button = Button(label='SHOW LABELS',
                height_policy='fit',)
                #button_type="success")
    
# this code is called when the labels_button object is clicked
labels_button_code = """ 
                
        if (labels_button.label.localeCompare('SHOW LABELS') == 0) {
            labels_button.label = 'HIDE LABELS';
            labels_1.visible = true;
            labels_2.visible = true;
        }
        
        else if (labels_button.label.localeCompare('HIDE LABELS') == 0) {
            labels_button.label = 'SHOW LABELS';
            labels_1.visible = false;
            labels_2.visible = false;
        }
        

        
        labels_1.change.emit();
        labels_1.change.emit();
        labels_button.change.emit();


    """

labels_button.js_on_click(CustomJS(args=dict(labels_1 = labels_1,
                                             labels_2 = labels_2,
                                             labels_button = labels_button),
                                   code=labels_button_code ))

###################
# multichoice box #
###################

OPTIONS = [str(n) for n in G1.nodes]

multi_choice = MultiChoice(value=[], options=OPTIONS,
                           max_height=80,
                           height_policy="fit")

# this code is called when the color_button object is clicked
color_button_code = """ 
        
        var node_data_1 = gr_1.node_renderer.data_source.data;
        var node_data_2 = gr_2.node_renderer.data_source.data;
        
        var selected_nodes = multi_choice.value.map(Number);

        // color selected nodes of 1 to red, not selected to white
        for (var i = 0; i < node_data_1['index'].length; i++) {
            // node indices starts from 0
            if (selected_nodes.includes(i)) {
               gr_1.node_renderer.data_source.data['color'][i] = color_list[color_list.length - 1];
            } 
            else {
               gr_1.node_renderer.data_source.data['color'][i] = color_list[0];
            }
        }      
        
        // get list of points in selected_nodes
        var points_in_selected_nodes = new Array();
        for (var i = 0; i < selected_nodes.length; i++) {
            points_in_selected_nodes.push(...gr_1.node_renderer.data_source.data['points covered'][selected_nodes[i]]);
        }
                
        // select only unique values, convert it to a set
        points_in_selected_nodes = new Set(points_in_selected_nodes);
        
        // START double points
        // uncomment for Alexander to Jones
        
        points_in_selected_nodes = Array.from(points_in_selected_nodes);
        console.log(points_in_selected_nodes.length);
        var double_points = new Array();
        for (var i = 0; i < points_in_selected_nodes.length; i++) {
            double_points.push(2*points_in_selected_nodes[i]);
            double_points.push(2*points_in_selected_nodes[i] + 1);
        }
        
        points_in_selected_nodes = new Set(double_points);
        
        // END double points

        // color nodes in G2 according to the percentage of points that are in POINTS_IN_SELECTED_NODES
        for (var i = 0; i < node_data_2['index'].length; i++) {
            
            const points_2 = new Set(node_data_2['points covered'][i]);
            
            const intersection = new Set([...points_in_selected_nodes].filter(value=>points_2.has(value)));
  
            var coverage = intersection.size / points_2.size ;
            
            gr_2.node_renderer.data_source.data['coverage'][i] = coverage;
            gr_2.node_renderer.data_source.data['color'][i] = color_list[Math.round(coverage*100)];
        }      
        
        gr_1.node_renderer.data_source.change.emit();
        gr_2.node_renderer.data_source.change.emit();


    """

color_button.js_on_click(CustomJS(args=dict(gr_1 = graph_renderer_1,
                                            gr_2 = graph_renderer_2,
                                            multi_choice = multi_choice,
                                            color_list = color_list),
                                  code=color_button_code ))

# this code is called when nodes on the left are selected
tap_code = """ 
        
        console.log(chi);
        
        var nodes_clicked_int = gr_1.node_renderer.data_source.selected.indices;
                
        var nodes_clicked_str = new Array();
        for (var i = 0; i < nodes_clicked_int.length; i++){
            nodes_clicked_str.push(nodes_clicked_int[i].toString());
        }
        
        multi_choice.value = [...new Set([...multi_choice.value, ...nodes_clicked_str])];

    """

# plot1.js_on_event(Tap, CustomJS(args=dict(gr_1 = graph_renderer_1,
#                                           multi_choice = multi_choice,
#                                           chi='Tap'),
#                                 code=tap_code ))


plot1.js_on_event(SelectionGeometry, CustomJS(args=dict(gr_1 = graph_renderer_1,
                                                        multi_choice = multi_choice,
                                                        chi='Selection'),
                                              code=tap_code,
                                              ))

##########
# LAYOUT #
##########
layout = grid([[labels_button,color_button], [multi_choice], [plot1, plot2]])



In [ ]:
# show on browser
show(layout) 

In [ ]:
## save to HTML file
#save(layout,'output/{}.html'.format(OUTPUT_PATH))